In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))
    elif erp_path == 'ern':
        epochs = epochs.apply_baseline(baseline = (-1.35, -1.05))

    return epochs, pp_directory

In [3]:
#separate function to get indexed epochs for random, since we have them before rotation or mirror
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

    
    return(triallist)
    

In [4]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [5]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, task, training):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s_%s-epo.fif' % (pp_num, condname, task, training))
    data.save(out_fname, overwrite = True)

In [6]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, task, training):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s_%s-ave.fif' % (pp_num, condname, task, training))
    evoked.save(out_fname, overwrite = True)

In [7]:
#save as evoked objects
#each perturbation condition can have separate loops
l_cutoff = 30

b0 = list(range(0,48))

# ALIGNED
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    aldata = epoch_trials('aln', pp, data, b0)
    #print(pp, numtrials)
    output_condition_epochs(pp, aldata, ppdir, 'P3', 'aligned', 'all')
    #filter at 30 Hz
    aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
    #then average across trials
    output_condition_evoked(pp, aldata, ppdir, 'P3', 'aligned', 'all')

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\mra_p001_run0-epo.fif ...


[Parallel(n_jobs=1)]: Done 3072 out of 3072 | elapsed:    1.0s finished


    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\frn\mra_p031_run0-epo.fif ...


[Parallel(n_jobs=1)]: Done 3072 out of 3072 | elapsed:    0.9s finished


    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 3072 out of 3072 | elapsed:    0.9s finished


In [8]:
#get grand averages - ALIGNED
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
#aligned

flist = []
evoked_list = []
for pp in participants:
    id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    pp_directory = os.path.join(id_directory, erps)
    fname = os.path.join(pp_directory, 'p%03d_%s_%s_%s-ave.fif' % (participants[pp], 'P3', 'aligned', 'all'))
    evoked = mne.read_evokeds(fname)
    flist.append(evoked)
    evoked = flist[pp][0]
    evoked_list.append(evoked)
    aln_b0_grand = mne.grand_average(evoked_list)
    aln_b0_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_P3_aligned_all-ave.fif ...


C:\Users\Raphael\AppData\Local\Temp\ipykernel_14108\4049494703.py:18: RuntimeWarning: Only a single dataset was passed to mne.grand_average().
  aln_b0_grand = mne.grand_average(evoked_list)


    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p003\frn\p003_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matr

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\frn\p020_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\p021_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:

In [9]:
#save as evoked objects
l_cutoff = 30

#ROTATED and MIRROR TRAINING
tasks = ['rot', 'mir']

#split into training blocks
#18 trials per block -> 5 blocks
b0 = list(range(0,18))
b1 = list(range(18,36))
b2 = list(range(36,54))
b3 = list(range(54,72))
b4 = list(range(72,90))
#b5 = list(range(60,72))
#b6 = list(range(72,90))
#training = [b0, b1, b2, b3, b4, b5, b6]
training = [b0, b1, b2, b3, b4]

       
#rot and mir
for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for t in range(0, len(training)):
            pdata = epoch_trials(tasks[task], pp, data, training[t])
            output_condition_epochs(pp, pdata, ppdir, 'P3', tasks[task], t)
            #filter at 30 Hz
            pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
            #then average across trials
            output_condition_evoked(pp, pdata, ppdir, 'P3', tasks[task], t)

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1152 out of 1152 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



...

[Parallel(n_jobs=1)]: Done 1152 out of 1152 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1152 out of 1152 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1152 out of 1152 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1152 out of 1152 | elapsed:    0.3s finished


In [10]:
#get grand averages - ROTATED AND MIRROR
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
tasks = ['rot', 'mir']
training = [0, 1, 2, 3, 4]
#rot and mir
for task in range(0, len(tasks)):
    for t in range(0, len(training)):
        flist = []
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_%s_%s_%s-ave.fif' % (participants[pp], 'P3', tasks[task], training[t]))
            if os.path.exists(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            flist.append(evoked)
            evoked = evoked[0]
            evoked_list.append(evoked)
            if task == 0 and t == 0:
                rot_b0_grand = mne.grand_average(evoked_list)
                rot_b0_flist = evoked_list
            elif task == 0 and t == 1:
                rot_b1_grand = mne.grand_average(evoked_list)
                rot_b1_flist = evoked_list
            elif task == 0 and t == 2:
                rot_b2_grand = mne.grand_average(evoked_list)
                rot_b2_flist = evoked_list
            elif task == 0 and t == 3:
                rot_b3_grand = mne.grand_average(evoked_list)
                rot_b3_flist = evoked_list
            elif task == 0 and t == 4:
                rot_b4_grand = mne.grand_average(evoked_list)
                rot_b4_flist = evoked_list
            elif task == 1 and t == 0:
                mir_b0_grand = mne.grand_average(evoked_list)
                mir_b0_flist = evoked_list
            elif task == 1 and t == 1:
                mir_b1_grand = mne.grand_average(evoked_list)
                mir_b1_flist = evoked_list
            elif task == 1 and t == 2:
                mir_b2_grand = mne.grand_average(evoked_list)
                mir_b2_flist = evoked_list
            elif task == 1 and t == 3:
                mir_b3_grand = mne.grand_average(evoked_list)
                mir_b3_flist = evoked_list
            elif task == 1 and t == 4:
                mir_b4_grand = mne.grand_average(evoked_list)
                mir_b4_flist = evoked_list
            

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_P3_rot_0-ave.fif ...
    Found the data of interest:


C:\Users\Raphael\AppData\Local\Temp\ipykernel_14108\1560354505.py:24: RuntimeWarning: Only a single dataset was passed to mne.grand_average().
  rot_b0_grand = mne.grand_average(evoked_list)


        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p003\frn\p003_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p004\frn\p004_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect ty

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\p021_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\frn\p022_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p023\frn\p023_P3_rot_0-ave.fif ...
    Found the data of interest:
    

C:\Users\Raphael\AppData\Local\Temp\ipykernel_14108\1560354505.py:27: RuntimeWarning: Only a single dataset was passed to mne.grand_average().
  rot_b1_grand = mne.grand_average(evoked_list)


...

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\frn\p007_P3_mir_3-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\frn\p008_P3_mir_3-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices availab

Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p025\frn\p025_P3_mir_3-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\frn\p026_P3_mir_3-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\frn\p027_P3_mir_3-ave.fif ...
    

C:\Users\Raphael\AppData\Local\Temp\ipykernel_14108\1560354505.py:51: RuntimeWarning: Only a single dataset was passed to mne.grand_average().
  mir_b4_grand = mne.grand_average(evoked_list)


        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\frn\p006_P3_mir_4-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\frn\p007_P3_mir_4-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect ty

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p024\frn\p024_P3_mir_4-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p025\frn\p025_P3_mir_4-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\frn\p026_P3_mir_4-ave.fif ...
    Found the data of interest:
    

In [11]:
#save as evoked objects
l_cutoff = 30

#RANDOM

#split into training blocks
b0 = list(range(0,12))
b1 = list(range(12,24))
b2 = list(range(24,36))
b3 = list(range(36,48))

training = [b0, b1, b2, b3]
#training = [b0]
            
# RANDOM (could be optimized, but in case we want to separate rdm before rot or mir, we have it as below)
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for t in range(0, len(training)):
        #rdmrot
        rdmrot_triallist = trial_list_rdm('rdmrot', pp, training[t])
        #rdmmir
        rdmmir_triallist = trial_list_rdm('rdmmir', pp, training[t])
    
        #combine both rdmrot and rdmmir accdg to conditions
        triallist_rdm = rdmrot_triallist + rdmmir_triallist
        triallist_rdm.sort()
        #save as one file of evokeds called rdm
        pdata = data[triallist_rdm]

        output_condition_epochs(pp, pdata, ppdir, 'P3', 'rdm', t)
        #filter at 30 Hz
        pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, pdata, ppdir, 'P3', 'rdm', t)

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1536 out of 1536 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



...

[Parallel(n_jobs=1)]: Done 1536 out of 1536 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1536 out of 1536 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1536 out of 1536 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1536 out of 1536 | elapsed:    0.4s finished


In [12]:
#get grand averages - RANDOM
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
tasks = ['rdm']
training = [0, 1, 2, 3]

#rdm
for task in range(0, len(tasks)):
    for t in range(0, len(training)):
        flist = []
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_%s_%s_%s-ave.fif' % (participants[pp], 'P3', tasks[task], training[t]))
            if os.path.exists(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            flist.append(evoked)
            evoked = evoked[0]
            evoked_list.append(evoked)
            if task == 0 and t == 0:
                rdm_b0_grand = mne.grand_average(evoked_list)
                rdm_b0_flist = evoked_list
            elif task == 0 and t == 1:
                rdm_b1_grand = mne.grand_average(evoked_list)
                rdm_b1_flist = evoked_list
            elif task == 0 and t == 2:
                rdm_b2_grand = mne.grand_average(evoked_list)
                rdm_b2_flist = evoked_list
            elif task == 0 and t == 3:
                rdm_b3_grand = mne.grand_average(evoked_list)
                rdm_b3_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_P3_rdm_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_P3_rdm_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_P3_rdm_0-ave.fif ...
    Found the data of interest:
    

C:\Users\Raphael\AppData\Local\Temp\ipykernel_14108\4112723554.py:25: RuntimeWarning: Only a single dataset was passed to mne.grand_average().
  rdm_b0_grand = mne.grand_average(evoked_list)


Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p004\frn\p004_P3_rdm_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p005\frn\p005_P3_rdm_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\frn\p006_P3_rdm_0-ave.fif ...
    

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p023\frn\p023_P3_rdm_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p024\frn\p024_P3_rdm_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\S

C:\Users\Raphael\AppData\Local\Temp\ipykernel_14108\4112723554.py:28: RuntimeWarning: Only a single dataset was passed to mne.grand_average().
  rdm_b1_grand = mne.grand_average(evoked_list)


...

Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\frn\p006_P3_rdm_2-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\frn\p007_P3_rdm_2-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\frn\p008_P3_rdm_2-ave.fif ...
    

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p025\frn\p025_P3_rdm_2-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\frn\p026_P3_rdm_2-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\S

C:\Users\Raphael\AppData\Local\Temp\ipykernel_14108\4112723554.py:34: RuntimeWarning: Only a single dataset was passed to mne.grand_average().
  rdm_b3_grand = mne.grand_average(evoked_list)


        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\frn\p006_P3_rdm_3-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\frn\p007_P3_rdm_3-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect ty

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p024\frn\p024_P3_rdm_3-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p025\frn\p025_P3_rdm_3-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\frn\p026_P3_rdm_3-ave.fif ...
    Found the data of interest:
    

In [14]:
#convert evokeds to data frame, so we can plot the ERPs better

flists = ['aln', 
          'rot_b0', 'rot_b1', 'rot_b2', 'rot_b3', 'rot_b4',
          'mir_b0', 'mir_b1', 'mir_b2', 'mir_b3', 'mir_b4',
          'rdm_b0', 'rdm_b1', 'rdm_b2', 'rdm_b3']

for f in flists:
    newdf = pd.DataFrame()
    for pp in participants:
        if f == 'aln':
            df = aln_b0_flist[pp].to_data_frame()
        elif f == 'rot_b0':
            df = rot_b0_flist[pp].to_data_frame()
        elif f == 'rot_b1':
            df = rot_b1_flist[pp].to_data_frame()
        elif f == 'rot_b2':
            df = rot_b2_flist[pp].to_data_frame()
        elif f == 'rot_b3':
            df = rot_b3_flist[pp].to_data_frame()
        elif f == 'rot_b4':
            df = rot_b4_flist[pp].to_data_frame()
        elif f == 'mir_b0':
            df = mir_b0_flist[pp].to_data_frame()
        elif f == 'mir_b1':
            df = mir_b1_flist[pp].to_data_frame()
        elif f == 'mir_b2':
            df = mir_b2_flist[pp].to_data_frame()
        elif f == 'mir_b3':
            df = mir_b3_flist[pp].to_data_frame()
        elif f == 'mir_b4':
            df = mir_b4_flist[pp].to_data_frame()
        elif f == 'rdm_b0':
            df = rdm_b0_flist[pp].to_data_frame()
        elif f == 'rdm_b1':
            df = rdm_b1_flist[pp].to_data_frame()
        elif f == 'rdm_b2':
            df = rdm_b2_flist[pp].to_data_frame()
        elif f == 'rdm_b3':
            df = rdm_b3_flist[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'Fz', 'Cz', 'Pz']]
        #df = df[['C3', 'C4']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_P3.csv' % (f))
    newdf.to_csv(newdf_filename)

In [15]:
#plot FRN/ERN here using MNE functions
#channels = ['FCz', 'Fz']
channels = ['FCz', 'Fz', 'Cz', 'Pz']
#channels = ['C3']
#channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(aln = aln_b0_flist, 
               rot_b0 = rot_b0_flist,
               rot_b4 = rot_b4_flist, 
               mir_b0 = mir_b0_flist, 
               mir_b4 = mir_b4_flist,
               rdm_b0 = rdm_b0_flist,
               rdm_b3 = rdm_b3_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)

combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


[<Figure size 800x600 with 1 Axes>]